In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import random
import string
import hashlib
import json

#Loading original data
columns_name = ['id', 'date', 'latitude', 'longitude' ]

df = pd.read_csv('../Data_Anom/real_data_set.csv', names = columns_name, sep = '\t')
df['date'] = df['date'].str.slice(stop = 16)
df['date'] = pd.to_datetime(df['date'], format = "%Y-%m-%d %H:%M")
df['week'] = df['date'].dt.isocalendar().week

In [2]:
anon = pd.read_csv('../Data_Anom/pocochocambo_434', names = columns_name, sep = '\t')
anon = anon.loc[anon.id != 'DEL']
anon['date'] = anon['date'].str.slice(stop = 16)
anon['date'] = pd.to_datetime(anon['date'], format = "%Y-%m-%d %H:%M")
anon['week'] = anon['date'].dt.isocalendar().week
anon.latitude = anon.latitude.astype(float)
anon.longitude = anon.longitude.astype(float)

In [3]:
print(anon)

           id                date  latitude  longitude  week
0         296 2015-03-25 22:00:00      4.87      45.76    13
1         296 2015-03-29 21:00:00      4.87      45.76    13
3         135 2015-05-06 19:00:00      4.87      45.75    19
4         290 2015-05-02 12:00:00      4.87      45.75    18
5         290 2015-04-29 14:00:00      4.87      45.75    18
...       ...                 ...       ...        ...   ...
34551841  165 2015-03-18 22:00:00      4.87      45.76    12
34551845  258 2015-03-14 12:00:00      4.88      45.78    11
34551846  278 2015-03-11 20:00:00      4.59      45.82    11
34551847  308 2015-03-15 19:00:00      4.87      45.78    11
34551848  270 2015-04-22 03:00:00      4.61      45.83    17

[23486259 rows x 5 columns]


In [4]:
print(df)

           id                date  latitude  longitude  week
0           1 2015-03-04 00:35:00  4.870147  45.772140    10
1           1 2015-03-04 00:35:00  4.870218  45.772095    10
2           1 2015-03-04 00:35:00  4.870210  45.772072    10
3           1 2015-03-04 00:35:00  4.870210  45.772072    10
4           1 2015-03-04 00:35:00  4.870210  45.772072    10
...       ...                 ...       ...        ...   ...
34551844  110 2015-03-12 16:23:00  2.343094  48.891650    11
34551845  110 2015-03-12 16:23:00  2.343094  48.891650    11
34551846  110 2015-03-12 16:23:00  2.343094  48.891649    11
34551847  110 2015-03-12 16:23:00  2.343094  48.891649    11
34551848  110 2015-03-12 19:29:00  2.343127  48.891772    11

[34551849 rows x 5 columns]


In [5]:
#List of id origin, id anonym, group of id-week

id_week_ori = df.groupby('id')['week'].unique()
id_week_anon = anon.groupby('id')['week'].unique()

list_id_ori = df["id"].unique()
list_id_anon = anon["id"].unique()

print(np.sort(list_id_ori), len(list_id_ori))
print(np.sort(list_id_anon), len(list_id_anon))

[  1   2   4   5   6   7   8   9  11  13  14  15  16  17  18  21  23  24
  25  26  27  28  29  30  31  32  34  35  36  37  38  39  41  42  43  44
  48  49  50  51  52  53  54  55  58  59  60  62  63  65  66  67  68  69
  70  71  72  73  75  77  78  81  83  84  87  89  98 107 110] 69
['0' '1' '10' '100' '101' '102' '103' '104' '105' '106' '107' '108' '109'
 '11' '110' '111' '112' '113' '114' '115' '116' '117' '118' '119' '12'
 '120' '121' '122' '123' '124' '125' '126' '127' '128' '129' '13' '130'
 '131' '132' '133' '134' '135' '136' '137' '138' '139' '14' '140' '141'
 '142' '143' '144' '145' '146' '147' '148' '149' '15' '150' '151' '152'
 '153' '154' '155' '156' '157' '158' '159' '16' '160' '161' '162' '163'
 '164' '165' '166' '167' '168' '169' '17' '170' '171' '172' '173' '174'
 '175' '176' '177' '178' '179' '18' '180' '181' '182' '183' '184' '185'
 '186' '187' '188' '189' '19' '190' '191' '192' '193' '194' '195' '196'
 '197' '198' '199' '2' '20' '200' '201' '202' '203' '204' '205' '20

In [6]:
#Group id--week--moyen_latitude--moyen_longitude--mean_date--count_line--mean_hour

#Original Table
df['hour'] = df['date'].dt.hour
mean_hour = df.groupby(['id','week'])['hour'].mean()

mean_lat_ori = df.groupby(['id','week'])['latitude'].mean()
mean_long_ori = df.groupby(['id','week'])['longitude'].mean()

mean_date_ori = df.groupby(['id','week'])['date'].mean()
count_ligne_ori = df.groupby(['id','week'])['date'].count()
mean_hour_ori = df.groupby(['id','week'])['hour'].mean()

ori_mean = df[['id','week']].drop_duplicates().reset_index(drop=True)
ori_mean['mean_latitude'] = ori_mean.apply(lambda x: mean_lat_ori[x.id][x.week], axis=1)
ori_mean['mean_longitude'] = ori_mean.apply(lambda x: mean_long_ori[x.id][x.week], axis=1)
ori_mean['mean_date'] = ori_mean.apply(lambda x: mean_date_ori[x.id][x.week], axis=1)
ori_mean['count_line'] = ori_mean.apply(lambda x: count_ligne_ori[x.id][x.week], axis=1)
ori_mean['mean_hour'] = ori_mean.apply(lambda x: mean_hour[x.id][x.week], axis=1)

#Anonymisation Table
anon['hour'] = anon['date'].dt.hour
mean_lat_anon = anon.groupby(['id','week'])['latitude'].mean()
mean_long_anon = anon.groupby(['id','week'])['longitude'].mean()
mean_date_anon = anon.groupby(['id','week'])['date'].mean()
count_ligne_anon = anon.groupby(['id','week'])['date'].count()
mean_hour_anon = anon.groupby(['id','week'])['hour'].mean()

anon_mean = anon[['id','week']].drop_duplicates().reset_index(drop=True)
anon_mean['mean_latitude'] = anon_mean.apply(lambda x: mean_lat_anon[x.id][x.week] , axis=1)
anon_mean['mean_longitude'] = anon_mean.apply(lambda x: mean_long_anon[x.id][x.week], axis=1)
anon_mean['mean_date'] = anon_mean.apply(lambda x: mean_date_anon[x.id][x.week], axis=1)
anon_mean['count_line'] = anon_mean.apply(lambda x: count_ligne_anon[x.id][x.week], axis=1)
anon_mean['mean_hour'] = anon_mean.apply(lambda x: mean_hour_anon[x.id][x.week], axis=1)

In [7]:
ori_mean['new_id'] = ori_mean['id']
scale = anon.shape[0] / df.shape[0]
for i in list_id_ori:
    for w in id_week_ori[i]:
        list_distance = list() 
        mean_lat = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_latitude'].to_list()[0]
        mean_long = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_longitude'].to_list()[0]
        mean_date = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_date'].to_list()[0]
        number_line = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'count_line'].to_list()[0]
        mean_hour = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_hour'].to_list()[0]
        
        group_week = anon_mean.loc[anon_mean.week ==w,['id','week','mean_latitude','mean_longitude','count_line','mean_date','mean_hour']].reset_index(drop=True)
        for j in range(len(group_week)):
            distance = np.power((group_week.mean_latitude[j] - mean_lat)**2 + (group_week.mean_longitude[j] - mean_long)**2, 1/2)
            
            delta_ratio = np.power(abs((group_week.count_line[j]/anon.shape[0] - number_line/df.shape[0])),4)
            delta_time = np.power(abs((group_week.mean_date[j] - mean_date).total_seconds() / 3600),1)
            
            
            delta_hour = np.power(abs((group_week.mean_hour[j] - mean_hour)) ,1/2)
            
            list_distance.append(distance*delta_ratio*delta_hour*delta_time) #*delta_ratio*delta_time*distance*delta_hour
            
        index = list_distance.index(min(list_distance))
        l = id_week_anon[group_week.id[index]]# list of week have id anon_mean.id[index] 
        while ((w not in l)):
            list_distance[index] = 1000000000
            index = list_distance.index(min(list_distance).all())
            l = id_week_anon[group_week.id[index]]
        if (w in l):
            ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'new_id'] = group_week.id[index]
        
#print(ori_mean.head())
print(ori_mean.new_id.nunique())

result = {}
count2 = 0
for i in list_id_ori:
    result[str(i)] = {}
    for w in id_week_ori[i]:
        result[str(i)]['2015'+'-'+str(w)] = [str(ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'new_id'][count2])]
        count2 +=1
print(result)

383
{'1': {'2015-10': ['267'], '2015-11': ['239'], '2015-12': ['156'], '2015-13': ['268'], '2015-14': ['195'], '2015-15': ['7'], '2015-16': ['2'], '2015-17': ['407'], '2015-18': ['3'], '2015-19': ['240'], '2015-20': ['302']}, '2': {'2015-10': ['167'], '2015-11': ['373'], '2015-12': ['295'], '2015-13': ['326'], '2015-14': ['224'], '2015-15': ['118'], '2015-16': ['358'], '2015-17': ['352'], '2015-18': ['52'], '2015-19': ['208'], '2015-20': ['350']}, '107': {'2015-10': ['303'], '2015-11': ['230'], '2015-12': ['257']}, '98': {'2015-10': ['90'], '2015-11': ['94']}, '21': {'2015-10': ['433'], '2015-11': ['37']}, '5': {'2015-10': ['186'], '2015-11': ['145']}, '9': {'2015-10': ['45'], '2015-11': ['211'], '2015-12': ['156'], '2015-13': ['351'], '2015-14': ['23'], '2015-15': ['379'], '2015-16': ['57'], '2015-19': ['39']}, '6': {'2015-10': ['292'], '2015-11': ['429'], '2015-12': ['265'], '2015-13': ['242'], '2015-14': ['253'], '2015-15': ['356'], '2015-16': ['85']}, '7': {'2015-10': ['161'], '201

In [8]:
jsonData = json.dumps(result, indent=4)
print(jsonData)

{
    "1": {
        "2015-10": [
            "267"
        ],
        "2015-11": [
            "239"
        ],
        "2015-12": [
            "156"
        ],
        "2015-13": [
            "268"
        ],
        "2015-14": [
            "195"
        ],
        "2015-15": [
            "7"
        ],
        "2015-16": [
            "2"
        ],
        "2015-17": [
            "407"
        ],
        "2015-18": [
            "3"
        ],
        "2015-19": [
            "240"
        ],
        "2015-20": [
            "302"
        ]
    },
    "2": {
        "2015-10": [
            "167"
        ],
        "2015-11": [
            "373"
        ],
        "2015-12": [
            "295"
        ],
        "2015-13": [
            "326"
        ],
        "2015-14": [
            "224"
        ],
        "2015-15": [
            "118"
        ],
        "2015-16": [
            "358"
        ],
        "2015-17": [
            "352"
        ],
        "2015-18": [
     